In [1]:
%pip install torch torch_geometric pandas scikit-learn

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 5.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from sklearn.metrics import accuracy_score

class GNNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

dataset = Planetoid(root=".", name="Cora")
data = dataset[0]

input_dim = dataset.num_node_features
hidden_dim = 64
output_dim = dataset.num_classes

model = GNNModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    with torch.no_grad():
        out = model(data)
        preds = out[data.test_mask].argmax(dim=1)
        accuracy = accuracy_score(data.y[data.test_mask].cpu(), preds.cpu())
    return accuracy

epochs = 20
for epoch in range(1, epochs + 1):
    loss = train()
    accuracy = test()
    print(f"Epoch {epoch}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

final_accuracy = test()
print(f"Final Accuracy on Test Data: {final_accuracy:.4f}")

Epoch 1, Loss: 1.9450, Accuracy: 0.6410
Epoch 2, Loss: 1.6930, Accuracy: 0.7610
Epoch 3, Loss: 1.3926, Accuracy: 0.7900
Epoch 4, Loss: 1.0794, Accuracy: 0.7970
Epoch 5, Loss: 0.8109, Accuracy: 0.8080
Epoch 6, Loss: 0.5919, Accuracy: 0.8100
Epoch 7, Loss: 0.4187, Accuracy: 0.8120
Epoch 8, Loss: 0.2916, Accuracy: 0.8010
Epoch 9, Loss: 0.2017, Accuracy: 0.7920
Epoch 10, Loss: 0.1377, Accuracy: 0.7880
Epoch 11, Loss: 0.0922, Accuracy: 0.7920
Epoch 12, Loss: 0.0611, Accuracy: 0.7910
Epoch 13, Loss: 0.0412, Accuracy: 0.7890
Epoch 14, Loss: 0.0286, Accuracy: 0.7890
Epoch 15, Loss: 0.0203, Accuracy: 0.7900
Epoch 16, Loss: 0.0143, Accuracy: 0.7880
Epoch 17, Loss: 0.0100, Accuracy: 0.7840
Epoch 18, Loss: 0.0072, Accuracy: 0.7880
Epoch 19, Loss: 0.0054, Accuracy: 0.7880
Epoch 20, Loss: 0.0042, Accuracy: 0.7860
Final Accuracy on Test Data: 0.7860
